In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import ROOT, uproot
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import seaborn as sns
from sklearn.utils import class_weight

# import necessary keras modules/functions
import keras
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
from keras.layers import BatchNormalization, Activation, AveragePooling2D, Concatenate
from keras import regularizers
from keras.constraints import max_norm

from keras.callbacks import TensorBoard

from time import time

Welcome to JupyROOT 6.18/04


Using TensorFlow backend.
/home/bora/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bora/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bora/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bora/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
df_signal = uproot.open("/mnt/harddisk4/scratch/signal_dtG1_delphes.root")["outtree"].pandas.df("*")
df_dyjets = uproot.open("/mnt/harddisk4/scratch/dyjets_delphes.root")["outtree"].pandas.df("*")
df_wjets  = uproot.open("/mnt/harddisk4/scratch/wjets_delphes.root")["outtree"].pandas.df("*")
n_events_to_be_used = min(len(df_dyjets),len(df_signal),len(df_wjets))
print(len(df_dyjets),len(df_signal),len(df_wjets))
n_events_to_be_used

23304 515858 1173


1173

In [3]:
df_signal.columns

Index(['br_weight', 'br_njets', 'br_nleptons', 'br_nbjets', 'br_scalar_HT',
       'br_jet_pt_1', 'br_jet_pt_2', 'br_jet_pt_3', 'br_jet_pt_4', 'br_MET',
       'br_MET_Phi', 'br_sphericity', 'br_aplanarity', 'br_Fox_Wolfram_1',
       'br_Fox_Wolfram_2', 'br_Fox_Wolfram_3', 'br_Fox_Wolfram_4'],
      dtype='object')

In [4]:
branches = ['br_njets', 'br_nleptons', 'br_nbjets', 'br_scalar_HT',
       'br_jet_pt_1', 'br_jet_pt_2', 'br_jet_pt_3', 'br_jet_pt_4', 'br_MET',
       'br_MET_Phi', 'br_sphericity', 'br_aplanarity', 'br_Fox_Wolfram_1',
       'br_Fox_Wolfram_2', 'br_Fox_Wolfram_3', 'br_Fox_Wolfram_4']

In [5]:
df_signal = uproot.open("/mnt/harddisk4/scratch/signal_dtG1_delphes.root")["outtree"].pandas.df(branches, entrystop=n_events_to_be_used)
df_dyjets = uproot.open("/mnt/harddisk4/scratch/dyjets_delphes.root")["outtree"].pandas.df(branches, entrystop=n_events_to_be_used)
df_wjets  = uproot.open("/mnt/harddisk4/scratch/wjets_delphes.root")["outtree"].pandas.df(branches, entrystop=n_events_to_be_used)

In [6]:
x_signal = df_signal.values
y_signal = np.full(len(x_signal) , 1)
x_dyjets = df_dyjets.values
x_wjets  = df_wjets.values
x_bkg    = np.concatenate((x_dyjets, x_wjets), axis=0)
y_bkg    = np.full(len(x_bkg) , 0)

In [7]:
x = np.concatenate((x_signal, x_bkg), axis=0)
y = np.concatenate((y_signal, y_bkg), axis=0)

In [8]:
x_train_unscaled, x_test_unscaled, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20)
#y_train = keras.utils.to_categorical(y_train, 2)
#y_test  = keras.utils.to_categorical(y_test, 2)

In [9]:
scaler  = preprocessing.StandardScaler().fit(x)
x_train = scaler.transform(x_train_unscaled)
x_test  = scaler.transform(x_test_unscaled)

In [10]:
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1,   activation='sigmoid'))
model.compile(loss=keras.losses.mean_squared_error , optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [11]:
history = model.fit(x_train, y_train, epochs=20, batch_size=2**6,  verbose=1, validation_data=(x_test, y_test))


Train on 2815 samples, validate on 704 samples
Epoch 1/20
2815/2815 [==============================] - 1s 191us/step - loss: 0.1913 - accuracy: 0.6771 - val_loss: 0.1347 - val_accuracy: 0.8196
Epoch 2/20
2815/2815 [==============================] - 0s 90us/step - loss: 0.1102 - accuracy: 0.8597 - val_loss: 0.1016 - val_accuracy: 0.8651
Epoch 3/20
2815/2815 [==============================] - 0s 92us/step - loss: 0.0922 - accuracy: 0.8796 - val_loss: 0.0952 - val_accuracy: 0.8764
Epoch 4/20
2815/2815 [==============================] - 0s 91us/step - loss: 0.0882 - accuracy: 0.8828 - val_loss: 0.0943 - val_accuracy: 0.8750
Epoch 5/20
2815/2815 [==============================] - 0s 86us/step - loss: 0.0866 - accuracy: 0.8845 - val_loss: 0.0935 - val_accuracy: 0.8764
Epoch 6/20
2815/2815 [==============================] - 0s 88us/step - loss: 0.0855 - accuracy: 0.8856 - val_loss: 0.0934 - val_accuracy: 0.8764
Epoch 7/20
2815/2815 [==============================] - 0s 88us/step - loss: 0.08

2020-05-06 18:49:22.577157: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-05-06 18:49:22.589790: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-05-06 18:49:22.710285: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x92b72a0 executing computations on platform CUDA. Devices:
2020-05-06 18:49:22.710341: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2020-05-06 18:49:22.738165: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2600000000 Hz
2020-05-06 18:49:22.743922: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x890e930 executing computations on platform Host. Devices:
2020-05-06 18:49:22.743963: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefin

In [66]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 128)               2176      
_________________________________________________________________
dense_58 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 129       
Total params: 18,817
Trainable params: 18,817
Non-trainable params: 0
_________________________________________________________________
